In [1]:
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

from langchain.llms import HuggingFaceHub


from langchain.retrievers import BM25Retriever, EnsembleRetriever

import re

In [2]:
file_path = "C:\\Users\\mangl\\Downloads\\MeerTarbani.pdf"
data_file = UnstructuredPDFLoader(file_path)
docs = data_file.load()

In [3]:
splitter = RecursiveCharacterTextSplitter(chunk_size=200,
                                          chunk_overlap=100)
chunks = splitter.split_documents(docs)

In [4]:
embeddings = HuggingFaceEmbeddings(model_name ='BAAI/bge-large-zh-v1.5')

C:\Users\mangl\AppData\Roaming\Python\Python310\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
C:\Users\mangl\AppData\Roaming\Python\Python310\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    #model_kwargs={"temperature": 0.3,"max_new_tokens":250},
    huggingfacehub_api_token='hf_xxxxxxxx',
)

C:\Users\mangl\AppData\Roaming\Python\Python310\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 0.2.0. Use HuggingFaceEndpoint instead.
  warn_deprecated(


In [6]:
vectorstore = Chroma.from_documents(chunks, embeddings)

In [7]:
vectorstore_retreiver = vectorstore.as_retriever(search_kwargs={"k": 3})

In [8]:
keyword_retriever = BM25Retriever.from_documents(chunks)
keyword_retriever.k =  3

In [9]:
ensemble_retriever = EnsembleRetriever(retrievers=[vectorstore_retreiver,
                                                   keyword_retriever],
                                       weights=[0.5, 0.5])

In [10]:
templated = """
<|system|>>
From now onwards, you're Meer Tarbani. Below i've attached context of him and a query. Please answer the query in the context of Meer Tarbani.

CONTEXT: {context}
</s>
<|user|>
{query}
</s>
<|assistant|>
"""

In [11]:
prompt = ChatPromptTemplate.from_template(templated)
output_parser = StrOutputParser()


In [12]:
chain = (
    {"context": ensemble_retriever, "query": RunnablePassthrough()}
    | prompt
    | llm
    | output_parser
)

In [13]:
invoked = chain.invoke("Who is meer tarbani?")
pattern = r"<\|assistant\|>(.*)$"
match = re.search(pattern, invoked, re.DOTALL)
if match:
  # Extract the captured group (response content)
  response = match.group(1)
  print(response)
else:
  print("No response found from <|assistant|>")



Meer Tarbani is a dynamic Full Stack Web Developer with a Bachelor's degree in Computer Applications and Computer Science from Silver Oak University. He has practical experience as a tech support at GAOTek Inc. And as a Flipkart SCOA Intern. With a strong foundation in both academia and practical experience, Meer is poised to make meaningful contributions to the tech industry. You can connect with him via email at redskull@duck.com or
